# 🏥 Dự đoán Bệnh Tiểu Đường - Demo Ứng Dụng

## Phần 4: Dự đoán nguy cơ tiểu đường cho bệnh nhân
- Nhập thông tin sức khỏe
- Dự đoán xác suất mắc bệnh
- Phân loại mức rủi ro (Thấp, Trung bình, Cao)

In [5]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Xác định đường dẫn mô hình
model_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'models')

# Tải mô hình tốt nhất (Random Forest)
rf_model = joblib.load(os.path.join(model_dir, 'random_forest_model.pkl'))
scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
feature_names = joblib.load(os.path.join(model_dir, 'feature_names.pkl'))

print('✓ Tải mô hình Random Forest thành công')
print(f'✓ Các đặc trưng: {feature_names}')

✓ Tải mô hình Random Forest thành công
✓ Các đặc trưng: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


## 1️⃣ Hàm Dự Đoán Nguy Cơ

In [ ]:
def classify_risk(probability):
    """Phân loại mức rủi ro dựa trên xác suất"""
    if probability < 0.4:
        return 'Thấp', '🟢 Ít nguy cơ mắc tiểu đường'
    elif probability <= 0.7:
        return 'Trung bình', '🟡 Có đấu hiệu mắc tiểu đường'
    else:
        return 'Cao', '🔴 Nguy cơ cao mắc tiểu đường'

def predict_diabetes_risk(patient_data):
    """
    Dự đoán nguy cơ tiểu đường cho bệnh nhân
    
    patient_data: dict hoặc list chứa 8 giá trị đầu vào
    Returns: dict với thông tin dự đoán
    """
    # Chuyển thành numpy array
    if isinstance(patient_data, dict):
        data = np.array([patient_data[name] for name in feature_names]).reshape(1, -1)
    else:
        data = np.array(patient_data).reshape(1, -1)
    
    # Chuẩn hóa
    data_scaled = scaler.transform(data)
    
    # Dự đoán xác suất
    probability = rf_model.predict_proba(data_scaled)[0, 1]
    risk_level, risk_desc = classify_risk(probability)
    
    return {
        'probability': probability,
        'risk_level': risk_level,
        'risk_description': risk_desc
    }

print('✓ Hàm dự đoán đã sẵn sàng')

## 2️⃣ Ví Dụ Dự Đoán Cho Bệnh Nhân

In [ ]:
def display_prediction(patient_info, prediction_result):
    """Hiển thị kết quả dự đoán một cách đẹp mắt"""
    print('\n' + '='*70)
    print('🏥 KẾT QUẢ DỰ ĐOÁN NGUY CƠ TIỂU ĐƯỜNG')
    print('='*70)
    
    # Hiển thị thông tin bệnh nhân
    print('\n📋 Thông tin bệnh nhân:')
    for feature, value in patient_info.items():
        print(f'  {feature:.<30} {value}')
    
    # Hiển thị kết quả dự đoán
    print('\n🔍 Kết quả dự đoán:')
    print(f'  Xác suất mắc tiểu đường: {prediction_result["probability"]:.2%}')
    print(f'  Mức rủi ro: {prediction_result["risk_level"]}')
    print(f'  {prediction_result["risk_description"]}')
    print('\n' + '='*70)

# Ví dụ 1: Bệnh nhân nguy cơ thấp
print('\n🔹 VÍ DỤ 1: Bệnh nhân nguy cơ THẤP')
patient_1 = {
    'Pregnancies': 1,
    'Glucose': 89,
    'BloodPressure': 66,
    'SkinThickness': 23,
    'Insulin': 94,
    'BMI': 28.1,
    'DiabetesPedigreeFunction': 0.167,
    'Age': 21
}
result_1 = predict_diabetes_risk(patient_1)
display_prediction(patient_1, result_1)

In [ ]:
# Ví dụ 2: Bệnh nhân nguy cơ trung bình
print('\n🔹 VÍ DỤ 2: Bệnh nhân nguy cơ TRUNG BÌNH')
patient_2 = {
    'Pregnancies': 3,
    'Glucose': 125,
    'BloodPressure': 70,
    'SkinThickness': 26,
    'Insulin': 115,
    'BMI': 31.1,
    'DiabetesPedigreeFunction': 0.347,
    'Age': 35
}
result_2 = predict_diabetes_risk(patient_2)
display_prediction(patient_2, result_2)

In [ ]:
# Ví dụ 3: Bệnh nhân nguy cơ cao
print('\n🔹 VÍ DỤ 3: Bệnh nhân nguy cơ CAO')
patient_3 = {
    'Pregnancies': 6,
    'Glucose': 175,
    'BloodPressure': 72,
    'SkinThickness': 35,
    'Insulin': 148,
    'BMI': 38.5,
    'DiabetesPedigreeFunction': 0.605,
    'Age': 48
}
result_3 = predict_diabetes_risk(patient_3)
display_prediction(patient_3, result_3)

## 3️⃣ So Sánh Dự Đoán Giữa 4 Mô Hình

In [ ]:
# Tải tất cả các mô hình
models = {}
model_configs = {
    'Logistic Regression': 'logistic_regression_model.pkl',
    'Random Forest': 'random_forest_model.pkl',
    'XGBoost': 'xgboost_model.pkl',
    'KNN': 'knn_model.pkl'
}

for model_name, model_file in model_configs.items():
    model_path = os.path.join(model_dir, model_file)
    try:
        model = joblib.load(model_path)
        models[model_name] = model
        print(f'✓ Tải {model_name} thành công')
    except FileNotFoundError:
        print(f'⚠️  Không tìm thấy {model_name} tại {model_path}')

print(f'✓ Tổng cộng {len(models)} mô hình được tải')

In [ ]:
def compare_all_models(patient_data):
    """So sánh dự đoán giữa tất cả các mô hình"""
    
    # Chuyển thành numpy array
    if isinstance(patient_data, dict):
        data = np.array([patient_data[name] for name in feature_names]).reshape(1, -1)
    else:
        data = np.array(patient_data).reshape(1, -1)
    
    # Chuẩn hóa
    data_scaled = scaler.transform(data)
    
    results = []
    for model_name, model in models.items():
        probability = model.predict_proba(data_scaled)[0, 1]
        risk_level, _ = classify_risk(probability)
        results.append({
            'Model': model_name,
            'Probability': f'{probability:.2%}',
            'Risk Level': risk_level
        })
    
    return pd.DataFrame(results)

# So sánh cho bệnh nhân 3 (nguy cơ cao)
print('\n📊 SO SÁNH DỰ ĐOÁN GIỮA TẤT CẢ MÔ HÌNH')
print('='*70)
print(f'\nBệnh nhân: Glucose={patient_3["Glucose"]}, BMI={patient_3["BMI"]}, Age={patient_3["Age"]}')
print('\n')
comparison = compare_all_models(patient_3)
print(comparison.to_string(index=False))

## 4️⃣ Khuyến Nghị và Hành Động

In [ ]:
def get_recommendations(probability, patient_data):
    """Đưa ra khuyến nghị dựa trên mức rủi ro"""
    
    print('\n' + '='*70)
    print('💡 KHUYẾN NGỊ VÀ HÀNH ĐỘNG')
    print('='*70)
    
    risk_level, risk_desc = classify_risk(probability)
    
    if probability < 0.4:
        print('\n✓ NGUY CƠ THẤP:')
        print('- Duy trì lối sống lành mạnh')
        print('- Tập thể dục đều đặn (150 phút/tuần)')
        print('- Ăn uống cân bằng, tránh đường')
        print('- Kiểm tra định kỳ 2 năm/lần')
    elif probability <= 0.7:
        print('\n⚠️  NGUY CƠ TRUNG BÌNH:')
        print('- Cần chú ý và tăng cường kiểm soát')
        print('- Giảm cân nếu BMI cao')
        print('- Tập thể dục 30 phút hàng ngày')
        print('- Kiểm tra hàng năm')
        print('- Tham khảo ý kiến bác sĩ về chế độ ăn')
    else:
        print('\n🔴 NGUY CƠ CAO:')
        print('- CẦN KIỂM TRA VÀ THĂM KHÁ BÁC SĨ NGAY')
        print('- Làm xét nghiệm glucose, HbA1c')
        print('- Có thể cần vào chế độ điều trị')
        print('- Giảm cân nhanh chóng')
        print('- Bắt đầu chương trình tập luyện')
        print('- Kiểm tra hàng 3 tháng')
    
    print('\n' + '='*70)

# Đưa ra khuyến nghị cho bệnh nhân 3
get_recommendations(result_3['probability'], patient_3)

## 5️⃣ Tóm Tắt Hệ Thống Dự Đoán

### 🎯 Các Tính Năng:
- ✅ Dự đoán xác suất mắc tiểu đường dựa trên 8 chỉ số sức khỏe
- ✅ Phân loại 3 mức rủi ro: Thấp, Trung bình, Cao
- ✅ So sánh dự đoán giữa 4 mô hình khác nhau
- ✅ Cung cấp khuyến nghị y tế cụ thể

### 📊 Hiệu Suất Mô Hình:
- Random Forest: Accuracy > 78%, F1-Score > 0.70, ROC-AUC > 0.84
- Đạt các mục tiêu đề ra

### 🏥 Ứng Dụng Thực Tiễn:
- Hỗ trợ bác sĩ sàng lọc bệnh nhân nguy cơ cao
- Tăng cường ý thức chăm sóc sức khỏe
- Phát hiện sớm bệnh tật

In [ ]:
print('\n' + '='*70)
print('✅ ĐỀ TÀI DỰ ĐOÁN BỆNH TIỂU ĐƯỜNG HOÀN THÀNH')
print('='*70)
print('\n📝 Tóm tắt quy trình:')
print('1. ✅ Tải và khám phá dữ liệu (EDA)')
print('2. ✅ Tiền xử lý dữ liệu')
print('3. ✅ Huấn luyện 4 mô hình ML')
print('4. ✅ Đánh giá hiệu suất mô hình')
print('5. ✅ Xây dựng ứng dụng demo dự đoán')
print('\n🎓 Các kỹ năng được học:')
print('- Xử lý dữ liệu (Data Preprocessing)')
print('- Xây dựng mô hình ML (Scikit-learn)')
print('- Đánh giá mô hình (Metrics & Evaluation)')
print('- Triển khai hệ thống dự đoán')
print('\n📚 Nguồn dữ liệu:')
print('- Pima Indians Diabetes Database (Kaggle)')
print('\n🔧 Công nghệ sử dụng:')
print('- Python 3.x')
print('- Pandas, NumPy, Scikit-learn')
print('- XGBoost, Matplotlib, Seaborn')
print('\n' + '='*70)

In [ ]:
print(\"\\n\" + \"=\"*70)
print(\"✅ ĐỀ TÀI DỰ ĐOÁN BỆNH TIỂU ĐƯỜNG HOÀN THÀNH\")
print(\"=\"*70)
print(\"\\n📝 Tóm tắt quy trình:\"
      \"\\n1. ✅ Tải và khám phá dữ liệu (EDA)\"
      \"\\n2. ✅ Tiền xử lý dữ liệu\"
      \"\\n3. ✅ Huấn luyện 4 mô hình ML\"
      \"\\n4. ✅ Đánh giá hiệu suất mô hình\"
      \"\\n5. ✅ Xây dựng ứng dụng demo dự đoán\"
      \"\\n\\n🎓 Các kỹ năng được học:\"
      \"\\n- Xử lý dữ liệu (Data Preprocessing)\"
      \"\\n- Xây dựng mô hình ML (Scikit-learn)\"
      \"\\n- Đánh giá mô hình (Metrics & Evaluation)\"
      \"\\n- Triển khai hệ thống dự đoán\"
      \"\\n\\n📚 Nguồn dữ liệu:\"
      \"\\n- Pima Indians Diabetes Database (Kaggle)\"
      \"\\n\\n🔧 Công nghệ sử dụng:\"
      \"\\n- Python 3.x\"
      \"\\n- Pandas, NumPy, Scikit-learn\"
      \"\\n- XGBoost, Matplotlib, Seaborn\"
      \"\\n\" + \"=\"*70
)

## 5️⃣ Tóm Tắt Hệ Thống Dự Đoán

### 🎯 Các Tính Năng:
- ✅ Dự đoán xác suất mắc tiểu đường dựa trên 8 chỉ số sức khỏe
- ✅ Phân loại 3 mức rủi ro: Thấp, Trung bình, Cao
- ✅ So sánh dự đoán giữa 4 mô hình khác nhau
- ✅ Cung cấp khuyến nghị y tế cụ thể

### 📊 Hiệu Suất Mô Hình:
- Random Forest: Accuracy > 78%, F1-Score > 0.70, ROC-AUC > 0.84
- Đạt các mục tiêu đề ra

### 🏥 Ứng Dụng Thực Tiễn:
- Hỗ trợ bác sĩ sàng lọc bệnh nhân nguy cơ cao
- Tăng cường ý thức chăm sóc sức khỏe
- Phát hiện sớm bệnh tật

In [2]:
def get_recommendations(probability, patient_data):
    \"\"\"Đưa ra khuyến nghị dựa trên mức rủi ro\"\"\"
    
    print(\"\\n\" + \"=\"*70)
    print(\"💡 KHUYẾN NGỊ VÀ HÀNH ĐỘNG\")
    print(\"=\"*70)
    
    risk_level, risk_desc = classify_risk(probability)
    
    if probability < 0.4:
        print(\"\\n✓ NGUY CƠ THẤP:\"
              \"\\n- Duy trì lối sống lành mạnh\"
              \"\\n- Tập thể dục đều đặn (150 phút/tuần)\"
              \"\\n- Ăn uống cân bằng, tránh đường\"
              \"\\n- Kiểm tra định kỳ 2 năm/lần\"
        )
    elif probability <= 0.7:
        print(\"\\n⚠️  NGUY CƠ TRUNG BÌNH:\"
              \"\\n- Cần chú ý và tăng cường kiểm soát\"
              \"\\n- Giảm cân nếu BMI cao\"
              \"\\n- Tập thể dục 30 phút hàng ngày\"
              \"\\n- Kiểm tra hàng năm\"
              \"\\n- Tham khảo ý kiến bác sĩ về chế độ ăn\"
        )
    else:
        print(\"\\n🔴 NGUY CƠ CAO:\"
              \"\\n- CẦN KIỂM TRA VÀ THĂM KHÁ BÁC SĨ NGAY\"
              \"\\n- Làm xét nghiệm glucose, HbA1c\"
              \"\\n- Có thể cần vào chế độ điều trị\"
              \"\\n- Giảm cân nhanh chóng\"
              \"\\n- Bắt đầu chương trình tập luyện\"
              \"\\n- Kiểm tra hàng 3 tháng\"
        )
    
    print(\"\\n\" + \"=\"*70)

# Đưa ra khuyến nghị cho bệnh nhân 3
get_recommendations(result_3['probability'], patient_3)

SyntaxError: unexpected character after line continuation character (3957123689.py, line 2)

## 4️⃣ Khuyến Nghị và Hành Động

In [ ]:
def compare_all_models(patient_data):
    \"\"\"So sánh dự đoán giữa tất cả các mô hình\"\"\"
    
    # Chuyển thành numpy array
    if isinstance(patient_data, dict):
        data = np.array([patient_data[name] for name in feature_names]).reshape(1, -1)
    else:
        data = np.array(patient_data).reshape(1, -1)
    
    # Chuẩn hóa
    data_scaled = scaler.transform(data)
    
    results = []
    for model_name, model in models.items():
        probability = model.predict_proba(data_scaled)[0, 1]
        risk_level, _ = classify_risk(probability)
        results.append({
            'Model': model_name,
            'Probability': f'{probability:.2%}',
            'Risk Level': risk_level
        })
    
    return pd.DataFrame(results)

# So sánh cho bệnh nhân 3 (nguy cơ cao)
print(\"\\n📊 SO SÁNH DỰ ĐOÁN GIỮA TẤT CẢ MÔ HÌNH\")
print(\"=\"*70)
print(f\"\\nBệnh nhân: Glucose={patient_3['Glucose']}, BMI={patient_3['BMI']}, Age={patient_3['Age']}\")
print(\"\\n\")
comparison = compare_all_models(patient_3)
print(comparison.to_string(index=False))

In [ ]:
# Tải tất cả các mô hình
models = {}
model_configs = {
    'Logistic Regression': 'logistic_regression_model.pkl',
    'Random Forest': 'random_forest_model.pkl',
    'XGBoost': 'xgboost_model.pkl',
    'KNN': 'knn_model.pkl'
}

for model_name, model_file in model_configs.items():
    model_path = os.path.join(model_dir, model_file)
    try:
        model = joblib.load(model_path)
        models[model_name] = model
        print(f"✓ Tải {model_name} thành công")
    except FileNotFoundError:
        print(f"⚠️  Không tìm thấy {model_name} tại {model_path}")

print(f"✓ Tổng cộng {len(models)} mô hình được tải")

## 3️⃣ So Sánh Dự Đoán Giữa 4 Mô Hình

In [ ]:
# Ví dụ 3: Bệnh nhân nguy cơ cao
print(\"\\n🔹 VÍ DỤ 3: Bệnh nhân nguy cơ CAO\")
patient_3 = {
    'Pregnancies': 6,
    'Glucose': 175,
    'BloodPressure': 72,
    'SkinThickness': 35,
    'Insulin': 148,
    'BMI': 38.5,
    'DiabetesPedigreeFunction': 0.605,
    'Age': 48
}
result_3 = predict_diabetes_risk(patient_3)
display_prediction(patient_3, result_3)

In [ ]:
# Ví dụ 2: Bệnh nhân nguy cơ trung bình
print(\"\\n🔹 VÍ DỤ 2: Bệnh nhân nguy cơ TRUNG BÌNH\")
patient_2 = {
    'Pregnancies': 3,
    'Glucose': 125,
    'BloodPressure': 70,
    'SkinThickness': 26,
    'Insulin': 115,
    'BMI': 31.1,
    'DiabetesPedigreeFunction': 0.347,
    'Age': 35
}
result_2 = predict_diabetes_risk(patient_2)
display_prediction(patient_2, result_2)

In [ ]:
def display_prediction(patient_info, prediction_result):
    \"\"\"Hiển thị kết quả dự đoán một cách đẹp mắt\"\"\"
    print(\"\\n\" + \"=\"*70)
    print(\"🏥 KẾT QUẢ DỰ ĐOÁN NGUY CƠ TIỂU ĐƯỜNG\")
    print(\"=\"*70)
    
    # Hiển thị thông tin bệnh nhân
    print(\"\\n📋 Thông tin bệnh nhân:\")
    for feature, value in patient_info.items():
        print(f\"  {feature:.<30} {value}\")
    
    # Hiển thị kết quả dự đoán
    print(\"\\n🔍 Kết quả dự đoán:\")
    print(f\"  Xác suất mắc tiểu đường: {prediction_result['probability']:.2%}\")
    print(f\"  Mức rủi ro: {prediction_result['risk_level']}\")
    print(f\"  {prediction_result['risk_description']}\")
    print(\"\\n\" + \"=\"*70)

# Ví dụ 1: Bệnh nhân nguy cơ thấp
print(\"\\n🔹 VÍ DỤ 1: Bệnh nhân nguy cơ THẤP\")
patient_1 = {
    'Pregnancies': 1,
    'Glucose': 89,
    'BloodPressure': 66,
    'SkinThickness': 23,
    'Insulin': 94,
    'BMI': 28.1,
    'DiabetesPedigreeFunction': 0.167,
    'Age': 21
}
result_1 = predict_diabetes_risk(patient_1)
display_prediction(patient_1, result_1)

## 2️⃣ Ví Dụ Dự Đoán Cho Bệnh Nhân

In [4]:
def classify_risk(probability):
    \"\"\"Phân loại mức rủi ro dựa trên xác suất\"\"\"
    if probability < 0.4:
        return 'Thấp', '🟢 Ít nguy cơ mắc tiểu đường'
    elif probability <= 0.7:
        return 'Trung bình', '🟡 Có đấu hiệu mắc tiểu đường'
    else:
        return 'Cao', '🔴 Nguy cơ cao mắc tiểu đường'

def predict_diabetes_risk(patient_data):
    \"\"\"
    Dự đoán nguy cơ tiểu đường cho bệnh nhân
    
    patient_data: dict hoặc list chứa 8 giá trị đầu vào
    Returns: dict với thông tin dự đoán
    \"\"\"
    # Chuyển thành numpy array
    if isinstance(patient_data, dict):
        data = np.array([patient_data[name] for name in feature_names]).reshape(1, -1)
    else:
        data = np.array(patient_data).reshape(1, -1)
    
    # Chuẩn hóa
    data_scaled = scaler.transform(data)
    
    # Dự đoán xác suất
    probability = rf_model.predict_proba(data_scaled)[0, 1]
    risk_level, risk_desc = classify_risk(probability)
    
    return {
        'probability': probability,
        'risk_level': risk_level,
        'risk_description': risk_desc
    }

print(\"✓ Hàm dự đoán đã sẵn sàng\")")

SyntaxError: unexpected character after line continuation character (624561998.py, line 2)

## 1️⃣ Hàm Dự Đoán Nguy Cơ

In [3]:
import sys
import os
sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# Xác định đường dẫn mô hình
model_dir = os.path.join(os.path.dirname(os.path.abspath('.')), 'models')

# Tải mô hình tốt nhất (Random Forest)
rf_model = joblib.load(os.path.join(model_dir, 'random_forest_model.pkl'))
scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
feature_names = joblib.load(os.path.join(model_dir, 'feature_names.pkl'))

print("✓ Tải mô hình Random Forest thành công")
print(f"✓ Các đặc trưng: {feature_names}")

✓ Tải mô hình Random Forest thành công
✓ Các đặc trưng: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


# 🏥 Dự đoán Bệnh Tiểu Đường - Demo Ứng Dụng

## Phần 4: Dự đoán nguy cơ tiểu đường cho bệnh nhân
- Nhập thông tin sức khỏe
- Dự đoán xác suất mắc bệnh
- Phân loại mức rủi ro (Thấp, Trung bình, Cao)